In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sources.data_pipeline import global_transformer, next_events, history
from sources.modelling import Model

In [13]:
df_test= pd.read_csv("database.nosync/updated_table.csv")
df_test.head()

,match_id,scheduled,player1_id,player2_id,winner_id,home_score,away_score
0,sr:match:41061433,2023-05-15T11:20:00+00:00,sr:competitor:407573,sr:competitor:254383,sr:competitor:254383,2.0,0.0
1,sr:match:41061401,2023-05-13T14:10:00+00:00,sr:competitor:407573,sr:competitor:16822,sr:competitor:407573,0.0,2.0
2,sr:match:40788805,2023-05-07T16:30:00+00:00,sr:competitor:46391,sr:competitor:407573,sr:competitor:407573,2.0,1.0
3,sr:match:40788801,2023-05-05T14:00:00+00:00,sr:competitor:64580,sr:competitor:407573,sr:competitor:407573,2.0,0.0
4,sr:match:40788873,2023-05-03T14:00:00+00:00,sr:competitor:90080,sr:competitor:407573,sr:competitor:407573,2.0,0.0


In [14]:
train_set = global_transformer(history())

In [15]:
train_set

,winner_is,player1_id,player2_id,atp_difference
0,1,1,89,88
1,1,1,82,81
2,1,24,1,-23
3,1,16,1,-15
4,1,11,1,-10
...,...,...,...,...
6995,0,353,30,-323
6996,0,353,352,-1
6997,0,353,44,-309
6998,0,2,353,351


In [16]:
rf= RandomForestClassifier()
data= train_set.drop(['winner_is'], axis=1)
target= train_set['winner_is']
X_train,X_test, y_train, y_test= train_test_split(data,target, test_size=0.2)
rf.fit(X_train, y_train)
y_pred=rf.predict(X_test)
acc=accuracy_score(y_test, y_pred)
print (acc)
cm= pd.crosstab(y_test, y_pred, rownames=['classe réelle'], colnames=['classe prédite'])
print(cm)


0.5604838709677419
classe prédite    0    1
classe réelle           
0               489  380
1               383  484


#retest avec fonction dans modelling, retrait en commentaire de l31 32 33 34 35, 46 à 49, 110 dans preprocessor.py pour régler le pb de competitors + modfcde la ligne 35 avec retrait de df_player1 et df_player2 et suppression de la colonne competitors dans planning_table.csv

In [21]:
mon_model= Model()
mon_model.modelling()

              precision    recall  f1-score   support

           0       0.57      0.58      0.57       870
           1       0.57      0.55      0.56       866

    accuracy                           0.57      1736
   macro avg       0.57      0.57      0.57      1736
weighted avg       0.57      0.57      0.57      1736


In [25]:
weekly_accuracy=mon_model.get_accuracy()
print(weekly_accuracy)

0.565668202764977


In [ ]:
# copie dans accuracy.txt
with open("accuracy.txt", "w") as f:
    f.write(str(weekly_accuracy))


In [30]:
if weekly_accuracy<0.55:
    print ("alert: need to fit the model again")